In [1]:
import pymorphy2
import string
import collections
from nltk import tokenize
from pathlib import Path
import numpy as np

In [2]:
morph = pymorphy2.MorphAnalyzer()

In [3]:
unlem_queries = [
    "Некоторые виды рыб размножаются не в воде, а на суше.",
    "Озеро Озеро — памятник природы.",
    "Канадский писатель стал первым лауреатом премии, названной в его честь."]

In [4]:
def get_doc_sentences(filename):
    with open(filename, "r", encoding="utf-8") as f:
        lines = f.readlines()
    sentences = []
    for line in lines:
        if line != "\n":
            sentences.extend(tokenize.sent_tokenize(line))
    return sentences

In [5]:
def get_lemmas(sentences):
    result = []
    for sent in sentences:
        sent = sent.translate(str.maketrans('', '', string.punctuation)).strip()
        splitted_sent = sent.split()
        lemmas = []
        for word in splitted_sent:
            p = morph.parse(word)[0]
            lemmas.append(p.normal_form)
        if lemmas:
            result.append(lemmas)
    return result

In [6]:
data_dir = Path.cwd() / "data"

docs = []
unlem_docs = []
for file in data_dir.iterdir():
    doc_sent = get_doc_sentences(data_dir / file)
    unlem_docs.extend(doc_sent)
    lemmas = get_lemmas(doc_sent)
    docs.extend(lemmas)
    
queries = get_lemmas(unlem_queries)

In [7]:
class Vectorizer:
    def __init__(self, docs, only_tf=False):
        self.docs = docs
        self.num_docs = len(docs)
        self.vocab = ["<unk>"] + list(set(word for doc in self.docs for word in doc))
        self.vocab_size = len(self.vocab)
        
        self.word2id = {word: idx for idx, word in enumerate(self.vocab)}
        self.only_tf = only_tf
        
    def compute_tf_idf(self):
        tf, df = np.zeros((self.num_docs, self.vocab_size), dtype=np.int), np.zeros(self.vocab_size, dtype=np.int)
        for i, doc in enumerate(self.docs):         
            v = np.zeros(self.vocab_size, dtype=np.int)
            uniq_words  = set(doc)
            for word in doc:
                v[self.word2id[word]] += 1
            
            for word in uniq_words:
                df[self.word2id[word]] += 1
            tf[i, :] = v
        df[0] = 1
        idf = np.log10(self.num_docs / df)
        # for <unk> word
        idf[0] = 0.0
        return tf, idf.reshape(1, -1)
    
    def fit_transform(self):
        self.tf, self.idf = self.compute_tf_idf()
        if self.only_tf:
            return self.tf
        tf_idf_scores = self.tf * self.idf
        return tf_idf_scores
    
    def transform(self, docs):
        scores = np.zeros((len(docs), self.vocab_size), dtype=np.float) 
        for i, doc in enumerate(docs):
            for word in doc:
                scores[i, self.word2id.get(word, 0)] += 1
        if not self.only_tf:
            scores = scores * self.idf
        return scores

In [8]:
vectorizer = Vectorizer(docs=docs)
docs_tfidf = vectorizer.fit_transform()
queries_tfidf = vectorizer.transform(queries)

In [10]:
docs_norm = np.linalg.norm(docs_tfidf, axis=1, keepdims=True)
queries_norm = np.linalg.norm(queries_tfidf, axis=1, keepdims=True)
dots = np.dot(docs_tfidf / docs_norm, (queries_tfidf / queries_norm).T)
dots.shape

(941, 3)

In [11]:
k = 15
top_ids = np.argpartition(dots, kth=range(-k,0), axis=0)[-k:, :][::-1, :]
col_mask = np.arange(len(queries)).reshape(1, -1).repeat(k, axis=0)
top_scores = dots[top_ids, col_mask]

In [12]:
for idx, q in enumerate(queries):
    print(f"#{idx + 1}: {unlem_queries[idx]}")
    top_docs, scores = top_ids[:, idx].tolist(), top_scores[:, idx].tolist()
    for d, s in zip(top_docs, scores):
        print(f"({s:.3f}): {unlem_docs[d]}")
    print("\n\n")

#1: Некоторые виды рыб размножаются не в воде, а на суше.
(0.394): Грунионы характеризуются необычным нерестовым поведением и являются одними из немногих видов рыб, которые откладывают икру не в воде, а на суше.
(0.370): С помощью гиногенеза размножаются некоторые популяции или виды рыб, представленные в природе одними самками.
(0.155): Виды хвостовых плавников рыб.
(0.147): У некоторых видов они участвуют в терморегуляции (термогенезе).
(0.144): У некоторых рыб могут отсутствовать брюшные, а в редких случаях — и грудные плавники.
(0.128): Рыбы живут в воде, которая занимает огромные пространства.
(0.127): Другими угрозами являются загрязнение воды, перегорождение рек, потепление, интродуцирование чужих видов, а также высыхание рек.
(0.125): Не характерная для рыб забота о потомстве наблюдается преимущественно у видов в приливно-отливной зоне, в узких заливах и бухтах, а также в реках и озёрах.
(0.123): Весь процесс занимает 30 секунд, некоторые особи могут оставаться вне воды в течени

In [13]:
vectorizer = Vectorizer(docs=docs, only_tf=True)
docs_tf = vectorizer.fit_transform()
queries_tf = vectorizer.transform(queries)

In [14]:
docs_norm = np.linalg.norm(docs_tf, axis=1, keepdims=True)
queries_norm = np.linalg.norm(queries_tf, axis=1, keepdims=True)
dots = np.dot(docs_tf / docs_norm, (queries_tf / queries_norm).T)

In [15]:
k = 15
top_ids = np.argpartition(dots, kth=range(-k,0), axis=0)[-k:, :][::-1, :]
col_mask = np.arange(len(queries)).reshape(1, -1).repeat(k, axis=0)
top_scores = dots[top_ids, col_mask]

In [16]:
for idx, q in enumerate(queries):
    print(f"#{idx + 1}: {unlem_queries[idx]}")
    top_docs, scores = top_ids[:, idx].tolist(), top_scores[:, idx].tolist()
    for d, s in zip(top_docs, scores):
        print(f"({s:.3f}): {unlem_docs[d]}")
    print("\n\n")

#1: Некоторые виды рыб размножаются не в воде, а на суше.
(0.552): Грунионы характеризуются необычным нерестовым поведением и являются одними из немногих видов рыб, которые откладывают икру не в воде, а на суше.
(0.423): С помощью гиногенеза размножаются некоторые популяции или виды рыб, представленные в природе одними самками.
(0.387): Многие виды рыб изменяют тип питания на протяжении жизни: например, в молодом возрасте питаются планктоном, а позже — рыбами или крупными беспозвоночными.
(0.385): Не характерная для рыб забота о потомстве наблюдается преимущественно у видов в приливно-отливной зоне, в узких заливах и бухтах, а также в реках и озёрах.
(0.381): Многие виды рыб содержатся в неволе — в аквариумах.
(0.354): Содержание и разведение аквариумных рыб в некоторых случаях помогает сохранить редкие их виды, вымирающие в исконных местах обитания.
(0.351): Гиногенез считается особой разновидностью партеногенеза, который в чистом виде у рыб не встречается.
(0.338): У некоторых рыб мо